In [72]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [73]:
forecast_df = pd.read_csv(r"C:\Users\olw05\shell_comp\waste_management\forecast_arima_2018-19.csv")

In [74]:
forecast_df.head()

,Index,Latitude,Longitude,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,0
0,0,24.66818,71.33144,8.475744,8.868568,9.202181,6.023070,10.788374,6.647325,7.387925,5.180296,7.483670,5.837380,NaN
1,1,24.66818,71.41106,24.029778,28.551348,25.866415,21.634459,34.419411,27.361908,40.431847,42.126945,47.519392,-5.522590,NaN
2,2,24.66818,71.49069,44.831635,66.111168,56.982258,53.003735,70.917908,42.517117,59.181629,73.203232,57.071377,23.667258,NaN
3,3,24.66818,71.57031,59.974419,80.821304,78.956543,63.160561,93.513924,70.203171,74.536720,101.067352,74.989417,31.017123,NaN
4,4,24.66818,71.64994,14.653370,19.327524,21.928144,17.899586,19.534035,19.165791,16.531315,26.086885,17.522164,8.424259,NaN


In [75]:
fig = px.scatter(forecast_df, x="Longitude", y="Latitude")
fig.show()

In [76]:
x = forecast_df.loc[:,["Longitude","Latitude"]].values
# print(len(x))

In [77]:
# from sklearn.cluster import KMeans
# wcss = []
# for i in range(10):
#   model = KMeans(n_clusters=i+1,init = "k-means++",random_state=0)
#   model.fit(x)
#   wcss.append(model.inertia_)
# # print(wcss)

In [78]:
# fig = px.line(x=[i+1 for i in range(10)], y=wcss, title='Within clusters sum of squares')
# fig.show()

In [79]:
k = 250
cluster = KMeans(n_clusters=k,init="k-means++",random_state=42)

x_means = cluster.fit_predict(x)

clusters = {}
for i in forecast_df["Index"]:
    clusters[i] = x_means[i]

# print(clusters)
# print(x_means)

C:\Users\olw05\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [80]:
df_cent = pd.DataFrame([[i, cluster.cluster_centers_[:,0][i],cluster.cluster_centers_[:,1][i]]
                       for i in range(k)],
                      columns=["cluster","x_loc","y_loc"])
df_cent.head()

,cluster,x_loc,y_loc
0,0,73.133863,20.792033
1,1,70.876437,23.534023
2,2,72.127693,22.798250
3,3,70.415750,21.242660
4,4,74.277567,23.136770


In [81]:
forecast_df["cluster"] = None

for i in clusters:
    forecast_df.loc[(forecast_df["Index"]==i),["cluster"]] = clusters[i]

In [82]:
fig = px.scatter(forecast_df, x="Longitude", y="Latitude", color="cluster")
# fig = go.Figure(data=go.Scatter(x=forecast_df["Longitude"], y=forecast_df["Latitude"], 
#                         marker_color=forecast_df["cluster"]))

# fig.add_trace(go.Scatter(x=df_cent["x_loc"], y=df_cent["y_loc"]))

fig.show()

In [83]:
from math import sin, cos, sqrt, atan2, radians
def calculate_dist(lat1,lon1,lat2,lon2):
    # Approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [84]:
forecast_df.head()

,Index,Latitude,Longitude,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,0,cluster
0,0,24.66818,71.33144,8.475744,8.868568,9.202181,6.023070,10.788374,6.647325,7.387925,5.180296,7.483670,5.837380,NaN,124
1,1,24.66818,71.41106,24.029778,28.551348,25.866415,21.634459,34.419411,27.361908,40.431847,42.126945,47.519392,-5.522590,NaN,124
2,2,24.66818,71.49069,44.831635,66.111168,56.982258,53.003735,70.917908,42.517117,59.181629,73.203232,57.071377,23.667258,NaN,222
3,3,24.66818,71.57031,59.974419,80.821304,78.956543,63.160561,93.513924,70.203171,74.536720,101.067352,74.989417,31.017123,NaN,222
4,4,24.66818,71.64994,14.653370,19.327524,21.928144,17.899586,19.534035,19.165791,16.531315,26.086885,17.522164,8.424259,NaN,222


In [85]:
demand_df = forecast_df[["Index","Latitude","Longitude","2018","2019","cluster"]]
demand_df = demand_df.merge(df_cent,on=["cluster"])
demand_df["centroid_dist"] = demand_df.apply(lambda x: calculate_dist(x["y_loc"],x["x_loc"],x["Latitude"],x["Longitude"]),axis=1)
demand_df.head()

,Index,Latitude,Longitude,2018,2019,cluster,x_loc,y_loc,centroid_dist
0,0,24.66818,71.33144,7.483670,5.837380,124,71.34139,24.577505,10.135823
1,1,24.66818,71.41106,47.519392,-5.522590,124,71.34139,24.577505,12.302475
2,14,24.58758,71.25181,0.424612,0.609925,124,71.34139,24.577505,9.129900
3,15,24.58758,71.33144,31.458012,7.614439,124,71.34139,24.577505,1.506230
4,16,24.58758,71.41106,107.777763,-13.401585,124,71.34139,24.577505,7.135549


In [86]:
near_centroid_df = demand_df.loc[demand_df[["Index","cluster","centroid_dist"]].groupby("cluster").centroid_dist.idxmin(),["Index","cluster"]]
near_centroid_dict = near_centroid_df.set_index('cluster').to_dict()
# near_centroid_dict["Index"][0]

In [87]:
agg_df = demand_df[["cluster","2018","x_loc","y_loc"]].groupby(["cluster","x_loc","y_loc"]).sum().reset_index()
agg_df_2 = demand_df[["cluster","2019"]].groupby(["cluster"]).sum().reset_index()
agg_df = agg_df.merge(agg_df_2,on=["cluster"])
agg_df["near_loc"] = agg_df["cluster"].apply(lambda x:near_centroid_dict["Index"][x])
agg_df.head()

,cluster,x_loc,y_loc,2018,2019,near_loc
0,0,73.133863,20.792033,1626.274944,-180.200975,2321
1,1,70.876437,23.534023,231.508951,68.442652,423
2,2,72.127693,22.798250,1654.696289,257.365322,1001
3,3,70.415750,21.242660,1236.402547,51.422908,2069
4,4,74.277567,23.136770,47.765405,12.166206,772


In [88]:
demand_df.to_csv(r"C:\Users\olw05\shell_comp\waste_management\cluster_demand_250.csv")

In [20]:
# result_df.loc[result_df["destination_index"].isna() == True]

In [340]:
result_df = pd.read_csv(r"C:\Users\olw05\shell_comp\waste_management\result_50_3.csv")
result_df.head()

,year,data_type,source_index,destination_index,value
0,20182019,depot_location,1970,NaN,NaN
1,20182019,depot_location,1073,NaN,NaN
2,20182019,depot_location,1222,NaN,NaN
3,20182019,depot_location,1199,NaN,NaN
4,20182019,depot_location,757,NaN,NaN


In [341]:
depot_loc = result_df.loc[result_df["data_type"]=="depot_location","source_index"].values
ref_loc = result_df.loc[result_df["data_type"]=="refinery_location","source_index"].values

biomass_ds = result_df.loc[result_df["data_type"]=="biomass_demand_supply",["source_index","destination_index"]].values

pellet_ds = result_df.loc[result_df["data_type"]=="pellet_demand_supply",["source_index","destination_index"]].values

In [342]:
fig = go.Figure(data = go.Scatter(x=forecast_df["Longitude"], y=forecast_df["Latitude"],opacity=0.2,mode="markers"))

fig.add_trace(go.Scatter(x=agg_df["x_loc"], y=agg_df["y_loc"], text=agg_df["near_loc"],
                mode="markers", name="Centroid", marker=dict(color='black')))

fig.add_trace(go.Scatter(x=agg_df.loc[agg_df["near_loc"].isin(depot_loc)==True,"x_loc"], 
                                   y=agg_df.loc[agg_df["near_loc"].isin(depot_loc)==True,"y_loc"], 
                        mode="markers",marker_color="blue", name="Depots"))

fig.add_trace(go.Scatter(x=agg_df.loc[agg_df["near_loc"].isin(ref_loc),"x_loc"],
                         y=agg_df.loc[agg_df["near_loc"].isin(ref_loc),"y_loc"], 
                        mode="markers",marker_color="red", name="Refinaries"))

fig.show()

In [343]:
fig = px.scatter(x=forecast_df["Longitude"], y=forecast_df["Latitude"],opacity=0.2,color=forecast_df["cluster"])

fig.add_scatter(x=agg_df["x_loc"], y=agg_df["y_loc"], text=agg_df["near_loc"],
                mode="markers", name="Centroid", marker=dict(color='black'))

fig.add_scatter(x=agg_df.loc[agg_df["near_loc"].isin(depot_loc)==True,"x_loc"], 
                 y=agg_df.loc[agg_df["near_loc"].isin(depot_loc)==True,"y_loc"],
               mode="markers", name="Depot", marker=dict(color='Blue'))

fig.add_scatter(x=agg_df.loc[agg_df["near_loc"].isin(ref_loc)==True,"x_loc"], 
                 y=agg_df.loc[agg_df["near_loc"].isin(ref_loc)==True,"y_loc"],
               mode='markers', name = "Refinaries", marker=dict(color='red'))

for i in range(len(biomass_ds)):
        fig.add_scatter(x=agg_df[agg_df["near_loc"].isin(biomass_ds[i])==True]["x_loc"], 
                      y=agg_df[agg_df["near_loc"].isin(biomass_ds[i])==True]["y_loc"], 
                        mode='lines', marker=dict(color='royalblue'))
        
for i in range(len(pellet_ds)):
        fig.add_scatter(x=agg_df[agg_df["near_loc"].isin(pellet_ds[i])==True]["x_loc"], 
                      y=agg_df[agg_df["near_loc"].isin(pellet_ds[i])==True]["y_loc"], 
                        mode='lines', marker=dict(color='darkred'))

fig.show()

In [344]:
result_df.loc[result_df["source_index"].isin([517,779,526,851,595,280,373,49,1501,2171,2038,2262,2300,2385])]["value"].sum()

44851.752242662704

In [345]:
0.2 * demand_df["2018"].sum()

65968.19950358423

In [270]:
result_df.loc[(result_df["data_type"]=="biomass_demand_supply") & (result_df["source_index"].isin(depot_loc)==False)]

,year,data_type,source_index,destination_index,value
50,2018,biomass_demand_supply,0,16.0,2120.29
69,2018,biomass_demand_supply,13,6.0,1422.78
70,2018,biomass_demand_supply,13,17.0,14226.04
72,2018,biomass_demand_supply,15,8.0,6408.90
73,2018,biomass_demand_supply,15,20.0,885.50
77,2018,biomass_demand_supply,19,7.0,12411.37
78,2018,biomass_demand_supply,19,18.0,4365.28
80,2018,biomass_demand_supply,21,9.0,2218.50
